# Lecture6 重回帰、statsmodelsにおけるカテゴリー変数の取り扱い

「Pythonで学ぶ計量経済学」より

https://py4etrics.github.io/13_Dummies.html

In [2]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
import wooldridge

In [3]:
!pip install wooldridge

In [4]:
import wooldridge

## ケース１：定数項だけの回帰分析
### ダミー変数なし
直感的な説明にするために説明変数が定数項だけの回帰分析から始める。 具体的には次の回帰式を考える。

$y=\beta_0 + \epsilon$

実は，この場合のOLS推定量
は被説明変数の平均と等しいことになる。この結果を確認するために以下ではwooldridgeパッケージのwage1のデータを使う。

In [5]:
wage1 = wooldridge.data('wage1')
wooldridge.data('wage1',description=True)

name of dataset: wage1
no of variables: 24
no of observations: 526

+----------+---------------------------------+
| variable | label                           |
+----------+---------------------------------+
| wage     | average hourly earnings         |
| educ     | years of education              |
| exper    | years potential experience      |
| tenure   | years with current employer     |
| nonwhite | =1 if nonwhite                  |
| female   | =1 if female                    |
| married  | =1 if married                   |
| numdep   | number of dependents            |
| smsa     | =1 if live in SMSA              |
| northcen | =1 if live in north central U.S |
| south    | =1 if live in southern region   |
| west     | =1 if live in western region    |
| construc | =1 if work in construc. indus.  |
| ndurman  | =1 if in nondur. manuf. indus.  |
| trcommpu | =1 if in trans, commun, pub ut  |
| trade    | =1 if in wholesale or retail    |
| services | =1 if in services indus.  

In [6]:
form_const = 'wage ~ 1'  # 定数項だけの場合は１が必要

res_const = ols(form_const, data=wage1).fit()
res_const.params

Intercept    5.896103
dtype: float64

この$\beta_0$は、平均と同じ

In [7]:
wage1['wage'].mean()

5.896102674787035

## ダミー変数あり：２つのケース
同じデータを使って$\{0,1\}$の値を取るダミー変数を考える。データセットwage1の中のfemaleという変数があり，以下の値を取る。

>女性の場合：female = 1</br>
>男性の場合：female = 0

値が０のカテゴリーを基本カテゴリーという。

$D$をfemaleのダミー変数とすると回帰式は以下のようになる。

$$y=\beta_0+\beta_1 D $$

さらに，この式はの$D=\{0,1\}$の値によって以下のように表すことができる。

男性：$D=0 \implies y=\beta_0 +\epsilon $

女性：$D=1 \implies y=\beta_0 +\beta_1 D+\epsilon $

即ち，OLS推定量は以下を表すことになる。

$\hat{\beta_0}$：男性の平均賃金

$\hat{\beta_0}+\hat{\beta_1}$：女性の平均賃金

この回帰式を使い，時間賃金の男女間の差について考察する

In [8]:
form_const_2 = 'wage ~ female'

res_const_2 = ols(form_const_2, data=wage1).fit()

res_const_2.params

Intercept    7.099489
female      -2.511830
dtype: float64

In [9]:
res_const_2.params.Intercept + res_const_2.params.female

4.587658740225293

というわけで、女性の時間賃金は約2.51ドル低い

## ダミー変数あり：４つのケース
データセットwage1にはmarriedという変数が含まれており，以下の値をとる。

> 既婚者の場合：married = 1</br>
> 未婚者の場合：married = 0

femaleと組み合わせることにより，次の４つのケースを分けることができる。

>未婚男性：female=0, married=0</br>
>未婚女性：female=1, married=0</br>
>既婚女性：female=1, married=1</br>
>既婚男性：female=0, married=1

この４つのケースを捉えるために、femaleとmarriedの値によって0もしくは1の値になるダミー変数を作成するが，２つのステップに分けて説明する。

### カテゴリ変数を無視して回帰を行った場合

In [10]:
form_const_3 = 'wage ~ female+married'

res_const_3 = ols(form_const_3, data=wage1).fit()

res_const_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wage   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     44.78
Date:                Sat, 23 Apr 2022   Prob (F-statistic):           1.12e-18
Time:                        10:25:35   Log-Likelihood:                -1391.5
No. Observations:                 526   AIC:                             2789.
Df Residuals:                     523   BIC:                             2802.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.1804      0.296     20.856      0.000       5.598       6.763
female        -2.2944      0.303     -7.582      0.000      -2.889      -1.700
married        1.3395      0.310      4.325      0.000       0.731       1.948
==============================================================================
Omnibus:                      229.596   Durbin-Watson:                   1.782
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              987.027
Skew:                           1.975   Prob(JB):                    4.67e-215
Kurtosis:                       8.425   Cond. No.                         3.73
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
print("singmale:",res_const_3.params.Intercept)
print("marmale:",res_const_3.params.Intercept+res_const_3.params.married)
print("singfem:",res_const_3.params.Intercept+res_const_3.params.female)
print("marfem:",res_const_3.params.Intercept+res_const_3.params.female+res_const_3.params.married)

singmale: 6.180429110566367
marmale: 7.51991011113031
singfem: 3.8860258351679904
marfem: 5.225506835731934


In [12]:
#答え合わせ独身男性の賃金平均
wage1.query('female==0 & married==0')['wage'].mean()

5.168023282705351

単純に重回帰しただけだと、答え合わせと合わない。

そこで、カテゴリー変数を導入する

$y= \beta_0+\beta_1 D_1 + \beta_2 D_2 + \beta_3 D_3$

- 基本カテゴリー：singmale 
- $D_1$=marmale  
- $D_2$=singfem
- $D_3$=marfem

$D_1=\{0,1\},D_2=\{0,1\},D_3=\{0,1\}$の取る値を考慮すると，以下の４つのパターンに分けることができる。


$D_1=0 \& D_2=0 \& D_3=0 \implies y=\beta_0+\epsilon$

$D_1=1 \& D_2=0 \& D_3=0 \implies y=\beta_0+\beta_1 +\epsilon$

$D_1=0 \& D_2=1 \& D_3=0 \implies y=\beta_0+\beta_2+\epsilon$

$D_1=0 \& D_2=0 \& D_3=1 \implies y=\beta_0+\beta_3+\epsilon$
 

即ち，OLS推定量は以下を表すことになる。

$\hat{\beta_0}$：未婚男性の平均賃金

$\hat{\beta_0}+\hat{\beta_1}$：既婚男性の平均賃金

$\hat{\beta_0}+\hat{\beta_2}$：未婚女性の平均賃金

$\hat{\beta_0}+\hat{\beta_3}$：既婚女性の平均賃金

In [13]:
# 以下では row をDataFrameの行と考える。

# 未婚男性の関数
def singmale(row):
    if row['female'] == 0 and row['married'] == 0:
        return 1
    else:
        return 0

# 既婚男性の関数
def marmale(row):
    if row['female'] == 0 and row['married'] == 1:
        return 1
    else:
        return 0

# 未婚女性の関数
def singfem(row):
    if row['female'] == 1 and row['married'] == 0:
        return 1
    else:
        return 0

# 既婚女性の関数
def marfem(row):
    if row['female'] == 1 and row['married'] == 1:
        return 1
    else:
        return 0

In [14]:
wage1.loc[:,'singmale'] = wage1.apply(singmale, axis=1)  # axis='columns'でもOK
wage1.loc[:,'marmale'] = wage1.apply(marmale, axis=1)    # axis='columns'でもOK
wage1.loc[:,'singfem'] = wage1.apply(singfem, axis=1)    # axis='columns'でもOK
wage1.loc[:,'marfem'] = wage1.apply(marfem, axis=1)      # axis='columns'でもOK

wage1.head(3)

,wage,educ,exper,tenure,nonwhite,female,married,numdep,smsa,northcen,...,profocc,clerocc,servocc,lwage,expersq,tenursq,singmale,marmale,singfem,marfem
0,3.10,11,2,0,0,1,0,2,1,0,...,0,0,0,1.131402,4,0,0,0,1,0
1,3.24,12,22,2,0,1,1,3,1,0,...,0,0,1,1.175573,484,4,0,0,0,1
2,3.00,11,2,0,0,0,0,2,0,0,...,0,0,0,1.098612,4,0,1,0,0,0


In [15]:
form_const_4 = 'wage ~ marmale + singfem + marfem'

res_const_4 = ols(form_const_4, data=wage1).fit()

res_const_4.params

Intercept    5.168023
marmale      2.815009
singfem     -0.556440
marfem      -0.602114
dtype: float64

In [16]:
#答え合わせ独身男性の賃金平均
wage1.query('female==0 & married==0')['wage'].mean()

5.168023282705351

In [17]:
#答え合わせ既婚男性の賃金平均
wage1.query('female==0 & married==1')['wage'].mean()

7.983031926002909

In [18]:
res_const_4.params.Intercept+res_const_4.params.marmale

7.98303192600291

## ケース２：定量的変数の導入
１つのダミー変数femaleだけが入るケースに次の変数を加えた回帰式を考える。

- educ：教育年数
- exper：雇用経験年数
- tenure：勤続年数

In [19]:
form_1 = 'wage ~ female + educ + exper+ tenure'

res_1 = ols(form_1, data=wage1).fit()

res_1.params

Intercept   -1.567939
female      -1.810852
educ         0.571505
exper        0.025396
tenure       0.141005
dtype: float64

賃金格差は約-1.81に減少した。これはeduc, exper, tenureの影響を取り除いた結果である。言い換えると，教育，経験，就労期間を所与とすると（それらの変数が同じである場合という意味），女性の時間賃金は約1.8ドル低い。

## ケース３：ダミー変数の交差項
ケース１と２の被説明変数はwageをそのまま使ったが，ここでは対数を取り賃金方程式にダミー変数の交差項を入れて考察する。

以下の回帰式を考える。

$$y=\beta_0+\beta_1 D+ \beta_2 Dx + \beta_3 x +\epsilon $$


ここで$D$がダミー変数，$x$は定量的変数であり，$Dx$がダミー変数の交差項である。ダミー変数が取る値$D=\{0,1\}$に分けて考えると，以
下を推定することになる。

$D=0 \implies y=\beta_0+\beta_3x+\epsilon$

$D=1 \implies y=(\beta_0+\beta_1)+(\beta_2+\beta_3)x+\epsilon$

具体例として $D$=female，$x$=educとするとOLS推定量は以下を表すことになる。

$\hat{\beta_0}$：（教育の効果を取り除いた）男性の平均賃金（対数）

$\hat{\beta_3}$：男性の賃金に対する教育の効果（％）

$\hat{\beta_0}+\hat{\beta_1}$：（教育の効果を取り除いた）女性の平均賃金（対数）

$\hat{\beta_2}+\hat{\beta_3}$：女性の賃金に対する教育の効果（％）


In [20]:
form_2 = 'np.log(wage) ~ female + female:educ + educ + exper + tenure'

res_2 = ols(form_2, data=wage1).fit()
res_2.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.4647,0.123,3.781,0.000,0.223,0.706
female,-0.2104,0.174,-1.209,0.227,-0.552,0.131
female:educ,-0.0072,0.014,-0.534,0.593,-0.034,0.019
educ,0.0903,0.009,10.359,0.000,0.073,0.107
exper,0.0046,0.002,2.850,0.005,0.001,0.008
tenure,0.0174,0.003,5.849,0.000,0.012,0.023


### t検定
- female
  - 教育などの影響を省いた後の平均賃金の差
  - 5%有意水準で$H_0$female=0は棄却できない。
- female:educ
  - 教育などの影響を省いた後の教育の収益率の差
  - 5%有意水準で$H_0$female:educ=0は棄却できない。

### F検定
$H_0$:female=female:educ=0の制約を考えよう

In [21]:
hypotheses = 'female=0, female:educ=0'

res_2.f_test(hypotheses).pvalue

3.894554691524031e-14

$H_0$は棄却される。

t検定では，femaleとfemale:educはそれぞれの帰無仮説が棄却されなかったが，F検定では制約が棄却された。一貫性がなく説明変数が不足している可能性がある。

# カテゴリー変数
カテゴリー変数とは定性的な変数であり，男女もカテゴリー変数の一種である。カテゴリー変数をダミー変数に変換するには2つの方法がある。

statsmodelsにはカテゴリー変数に自動的にダミー変数を割り当てる機能がある。操作は簡単で，単に回帰式の中でC()の中にカテゴリー変数を入れるだけである。

In [22]:
df = wage1.loc[:,['wage', 'female', 'educ']]

dfのメソッドreplace()を使ってfemaleの列の値を以下のルールに沿って変換し，それをdfにsexという列として入れ直す。

- $1 \implies$  female
- $0 \implies$ male

In [23]:
df.loc[:,'sex'] = df['female'].replace({1:'female',0:'male'})

In [24]:
df.head(3)

,wage,female,educ,sex
0,3.10,1,11,female
1,3.24,1,12,female
2,3.00,0,11,male


In [25]:
form_c = 'wage ~  C(sex) + educ'

res_c = ols(form_c, data=df).fit()

res_c.params

Intercept        -1.650545
C(sex)[T.male]    2.273362
educ              0.506452
dtype: float64

C(sex)[T.male]について

- TはTreatmentの頭文字で，通常のダミー変数を設定することを示している。
- maleはmaleの変数であることを表しており，自動的にfemaleが基本カテゴリーに設定されたことが分かる。

（結果）

C(sex)[T.male]はfemaleに比べてmaleの賃金は約2.27ドル高いことを示している。

## 蛇足
femaleとmarryedを独立に重回帰分析しても、答え合わせと合わなかったのは交絡因子をむししていたから。

In [28]:
form_const_3 = 'wage ~ female*married'

res_const_3 = ols(form_const_3, data=wage1).fit()

res_const_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wage   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.176
Method:                 Least Squares   F-statistic:                     38.45
Date:                Sat, 23 Apr 2022   Prob (F-statistic):           1.83e-22
Time:                        10:26:50   Log-Likelihood:                -1380.6
No. Observations:                 526   AIC:                             2769.
Df Residuals:                     522   BIC:                             2786.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          5.1680      0.361     14.299      0.000       4.458       5.878
female            -0.5564      0.474     -1.175      0.241      -1.487       0.374
married            2.8150      0.436      6.451      0.000       1.958       3.672
female:married    -2.8607      0.608     -4.708      0.000      -4.054      -1.667
==============================================================================
Omnibus:                      217.491   Durbin-Watson:                   1.773
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              877.781
Skew:                           1.881   Prob(JB):                    2.47e-191
Kurtosis:                       8.088   Cond. No.                         7.84
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""